# NH MU69 Rings Planning

## I start off here with a bunch of calculations for the MVIC outbound rings search.

## Initialize python

In [5]:
import astropy.units as u
import astropy.constants as c
import math
import astropy
import numpy as np
import matplotlib.pyplot as plt
import pymiecoated # Mie scattering library
import spiceypy as sp
from astropy.table import Table

import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
                                    # Ignore warnings caused by pymiecoated's numpy usage 

import matplotlib                   # Set latex to use *not* CMR.
matplotlib.rcParams['mathtext.fontset'] = 'stixsans'

r2d = 360 / (2*math.pi)             # Radians to degrees
d2r = 1/r2d

file_kernel = '/Users/throop/gv/dev/gv_kernels_new_horizons.txt'

## Calculate the Hill radius for a few bodies

$$r_H = a \sqrt[3]{m\over {3 M}}$$

## Set up some physical constants for NH and the encounter

In [6]:
a_pluto_ca   = 33*u.au             # NH heliocentric distance at Pluto flyby
a_mu69_ca    = 43*u.au             # NH heliocentric distance at MU69 flyby

dist_mu69_ca = 3046*u.km            # Distance from NHGV

rho_pluto       = 1.9 * u.gram/u.cm**3
rho_mu69        = 1.9 * u.gram/u.cm**3

a_hydra      = 65000 * u.km        # Hydra orbital distance around Pluto

fov_mvic     = 5.7 * d2r           # MVIC FOV, in radians
fov_lorri    = 0.3 * d2r           # LORRI FOV width, in radians 

resolution_mvic = fov_mvic / 5000   # Resolution in radians
resolution_lorri = fov_lorri / 1024 # Resolution in radians

r_mu69       = 16.5 * u.km
a_roche_mu69 = 2.5 * r_mu69

m_mu69 = 4/3 * math.pi * r_mu69**3 * rho_mu69
sp.furnsh(file_kernel)

In [7]:
### Calc NH velocity. This kernel doesn't have MU69 in it, so just use Pluto flyby. Velocities are about the same.

utc = '1 Jan 2016'
et = sp.utc2et(utc)
(st,lt) = sp.spkezr('Pluto', et, 'J2000', 'LT+S', 'New Horizons')
v_nh = sp.vnorm(st[3:6])*u.km/u.second

In [8]:
# Set up an astropy table with data for a few different planets

name = np.array(['Pluto', 'MU69', 'Saturn', 'Chariklo', 'Chiron'])
rho  = np.array([1.9,     rho_mu69, 1,        2.5,       2.5])*u.gram/u.cm**3
r    = np.array([1187,    r_mu69,  58232,    200,       109])*u.km
a    = np.array([33,      43,      10,       15.7,      13])*u.au

# Create the table

t = Table([name, rho, r, a], names=['Name', 'rho', 'radius', 'a'])

# Now do some math on the table. Note that for weird reasons, table columns' units get messed up if
# we just directly multiply them. We need to use .quantity to do it properly. This is documented
# but weird. See bottom of http://docs.astropy.org/en/stable/table/access_table.html

t['mass'] = (4/3. * math.pi * (t['radius'].quantity)**3 * t['rho'].quantity).to('kg')
t['a_hill']  = (a * (t['mass'].quantity/c.M_sun/3)**(1/3)).to('km')

# Add some more columns. 
t['a_hill / a_hydra'] = t['a_hill'].quantity / a_hydra   # Hill radius, in Hydra orbital radii

t['a_hill / r_<body>'] = t['a_hill'].quantity / t['radius'] # Hill radius, in body radii

t['t for MVIC halfwidth a_hill'] =  (t['a_hill'].quantity / (fov_mvic/2) / v_nh).to('day') 
                                                            # Time at which MVIC can see to Hill radius on each side

# Add some shortcuts to access individual rows. 
# Astropy does not allow e.g., t['Radius']['Saturn'] -- have to do t['Radius'][index_saturn]

index_pluto   = t['Name'] == 'Pluto'
index_mu69    = t['Name'] == 'MU69'
index_saturn  = t['Name'] == 'Saturn'
index_chariklo= t['Name'] == 'Chariklo'

# Set the column formatting

t['mass'].format = "%6.1e"
t['a_hill'].format = "%6.1f"
t['a_hill / a_hydra'].format = "%6.1f"
t['a_hill / r_<body>'].format = "%6.1f"
t['t for MVIC halfwidth a_hill'].format = "%6.1f"

ValueError: setting an array element with a sequence.

In [ ]:
t

So, a few interesting things from above. 

* MU69 rings go out to 2.0 a_hydra. That is the Hill radius.
* Pluto's hill radius goes out to 108 a_hydra. Much further.
* Roche radius is not plotted, but is usually ~2.5 r_body. So, 2000x closer in than a_hill.
* For Pluto, when we imaged at P+110d, that was out to about 0.95 R_hill on each side. But for MU69, we will do the same observation at P+2.2d.

Oh wow. So, at Pluto it made sense to search at P+100 days, because that was basically the Hill radius. But at MU69, the Hill radius is 50x smaller, so all the searching must be done much closer in.

At what point does MVIC see out to the Hill radius on each side of MU69?

### Where is the Phoebe ring, in Hill radii?

In [ ]:
a_ring_phoebe_inner = (50*t['radius'][index_saturn])
a_ring_phoebe_outer = (300*t['radius'][index_saturn])

a_ring_chiron = 324*u.km

a_ring_chariklo_a = 396*u.km
a_ring_chariklo_b = 405*u.km

In [ ]:
# Calculate Phoebe ring in Saturn Hill radii
# Strange astropy issue here: if we do np.array((a_inner, a_outer)) then the units are lost entirely! Weird.

print(a_ring_phoebe_inner / t['a_hill'][index_saturn].quantity)
print(a_ring_phoebe_outer / t['a_hill'][index_saturn].quantity)

In [ ]:
# Calculate Chariklo ring in Hill radii

print(a_ring_chariklo_b / t['a_hill'][index_chariklo].quantity)

Phoebe ring is at 0.2 R_Hill. I was concerned they were so far out that they were beyond the Hill radius. But no, not the case.

Also, don't confuse Hill radius with Roche radius. Rings are usually at Roche, but might be as far out as Hill, which is many many times further.

## Make a table that I will put into the NH MU69 planning wiki.

This should list resolution at a few different times.

In [ ]:
time = np.array([0, 0.5, 4, 50])*u.hour

In [ ]:
(t2['Time'].quantity * v_nh * resolution_mvic).to('km')

In [ ]:
t2 = Table([time], names=['Time'])

In [ ]:
t2['Distance']                = (t2['Time'].quantity     * v_nh + dist_mu69_ca).to('km')
t2['Resolution per MVIC pix'] = (t2['Distance'].quantity * resolution_mvic).to('km')
t2['Halfwidth km']            = (t2['Distance'].quantity * fov_mvic/2).to('km')
t2['Halfwidth a_hill_mu69']   = (t2['Distance'].quantity * fov_mvic/2 / t['a_hill'][index_mu69]).to('')
                               
t2['Resolution per MVIC pix'].format = "%6.2f"
t2['Halfwidth km'].format = "%6.1f"
t2['Halfwidth a_hill_mu69'].format = "%6.5f"
t2['Distance'].format = "%6.1f"

In [ ]:
t2

In [ ]:
# What is the Hill radius, in 

# Now, do some LORRI-specific calculations 

### To do: Make a table with LORRI resolution per pix. Just like MVIC, with same times as above. Put this into my LORRI MT.

In [ ]:
time_lorri = np.array([0, 0.5, 1, 4, 8, 50, 96])*u.hour

In [ ]:
t3 = Table([time_lorri], names=['Time'])

In [ ]:
# Dist from NH to MU69
t3['Distance']                = (t3['Time'].quantity     * v_nh + dist_mu69_ca).to('km')
t3['Resolution per LORRI pix']= (t3['Distance'].quantity * resolution_lorri).to('km')

# Width of LORRI at this distance (ie, how many km of orbital distance can we see)
t3['Width km']            = (t3['Distance'].quantity * fov_lorri).to('km')

# Width of LORRI at this distance (ie, how many hill radii can we see with LORRI)
t3['Width a_hill_mu69']   = (t3['Distance'].quantity * fov_lorri / t['a_hill'][index_mu69]).to('')

# How large a mosaic to see from 0 .. +1 r_hill (with no overlap)
t3['LORRI mosaic width a_hill_mu69'] = (1/t3['Width a_hill_mu69'].quantity) 
                               
t3['Resolution per LORRI pix'].format = "%6.2f"
t3['Width km'].format = "%6.1f"
t3['Width a_hill_mu69'].format = "%6.5f"
t3['Distance'].format = "%6.1f"

In [ ]:
t3

OK, so what is our observing plan?
* 4x4s, 10 sec. These were OK for Pluto outboudn and will be OK now. The resolution is not worth it, except maybe at the K+30m one. Do a 9x9 circle.  25 image at each footprint. We don't have a preferred position.
* 25 images at each footprint.
* At 30 min: requires a lot of images (9 x 5?). Better to do at t+1hr, out of way of MVIC and coma obs. Can do at lower res (0.27 km 1x1, 1 km 4x4). Mosaic will be easier to build. 
* T+4 hours. 4x4 mosaic. This will get us to -2000 .. 2000 km. [Actually, better to do 8 hrs... geometrically spaced.]
* T+100 hours. 5x5. (Or could do 5x1.) This will go 

## What is the orbital period at R_roche?

In [ ]:
p_roche = (2 * math.pi * a_roche_mu69) / np.sqrt(2 * c.G * m_mu69 / a_roche_mu69)

In [ ]:
p_roche.to('hour')